In [21]:
from nltk import ne_chunk, pos_tag, word_tokenize, StanfordNERTagger
from nltk.chunk import tree2conlltags
from os import walk, listdir
from os.path import isfile, join

In [22]:
#stanford NER configuration
st = StanfordNERTagger('/media/orithu/DATA/COMP34412/cwk2/stanford/classifiers/english.all.3class.distsim.crf.ser.gz',
					   '/media/orithu/DATA/COMP34412/cwk2/stanford/stanford-ner.jar',
					   encoding='utf-8')

chunk_collect = [] # save nltk NER result
stanf_collect = [] # save stanford NER result
for root, _, filenames in walk('corpus1'):
    for filename in filenames:
        f = open('corpus1/'+filename, "r", encoding='latin-1')
        s = f.read()
        tokenized = word_tokenize(s)
        # chunk NER
        chunk_result = tree2conlltags(ne_chunk(pos_tag(tokenized)))
        chunk_collect.extend(chunk_result)
        f = open('corpus1_tagged/nltk/'+filename, 'w')
        print(chunk_result, file=f)
        f.close()
        # stanford NER
        stanf_result = st.tag(tokenized)
        stanf_collect.extend(stanf_result)
        f = open('corpus1_tagged/stanford/'+filename, 'w')
        print(stanf_result, file=f)
        f.close()

In [23]:
print(len(chunk_collect))

145021


In [24]:
nl_list = [] # save NEs tagged with ORGANIZATUON by nltk NER
st_list = [] # save NEs tagged with ORGANIZATUON by stanford NER
nl_NE = []
st_NE = []
for i in range(len(chunk_collect)):
    if chunk_collect[i][2] == 'O' and stanf_collect[i][1] == 'O':
        if len(nl_NE) != 0 or len(st_NE) != 0:
            nl_list.append(nl_NE)
            st_list.append(st_NE)
            nl_NE = []
            st_NE = []
        continue
    
    if chunk_collect[i][2] == 'B-ORGANIZATION' or chunk_collect[i][2] == 'I-ORGANIZATION':
        nl_NE.append(chunk_collect[i][0])
    if stanf_collect[i][1] == 'ORGANIZATION':
        st_NE.append(stanf_collect[i][0])


print('%40s'%'nltk NER'+'%40s'%'stanford NER')
exact = 0
partial = 0
for i in range(len(nl_list)):
    print('%40s'%nl_list[i]+'%40s'%st_list[i])
    nl_token_len = len(nl_list[i])
    st_token_len = len(st_list[i])
    if nl_token_len == st_token_len:
        exact += 1
    elif nl_token_len != 0 and st_token_len != 0:
        partial += 1

print(f'chart size: {len(nl_list)}')
print(f"# of exact matching: {exact};# of partial matching: {partial}")

                                nltk NER                            stanford NER
                        ['Constitution']                                      []
                               ['Union']                                      []
                            ['Congress']                            ['Congress']
                            ['Congress']                            ['Congress']
                            ['Congress']                            ['Congress']
                           ['Territory']                                      []
                               ['Union']                                      []
                    ['Supreme', 'Court']                    ['Supreme', 'Court']
                           ['Territory']                                      []
                               ['Union']                                      []
                              ['States']                                      []
                            